In [1]:
# 导入必要的包
import jax
import jax.numpy as jnp

In [2]:
# 定义一个简单的函数，计算两个向量的卷积
x = jnp.arange(5)
w = jnp.array([2., 3., 4.])
print(f"x的形状是{x.shape}，w的形状是{w.shape}")

def convolve(x, w):
  output = []
  for i in range(1, len(x)-1):
    output.append(jnp.inner(x[i-1:i+2], w))
  return jnp.array(output)

convolve(x, w)

x的形状是(5,)，w的形状是(3,)


Array([11., 20., 29.], dtype=float32)

In [4]:
# 手动批处理计算卷积
xs = jnp.stack([x, x])
ws = jnp.stack([w, w])
print(f"xs的形状是{xs.shape}，ws的形状是{ws.shape}")

def manually_batched_convolve(xs, ws):
  output = []
  for i in range(xs.shape[0]):
    output.append(convolve(xs[i], ws[i]))
    #print(len(output), output[-1].shape)
  return jnp.stack(output)

manually_batched_convolve(xs, ws)

xs的形状是(2, 5)，ws的形状是(2, 3)


Array([[11., 20., 29.],
       [11., 20., 29.]], dtype=float32)

In [5]:
# 使用vmap自动批处理计算卷积
auto_batch_convolve = jax.vmap(convolve)
auto_batch_convolve(xs, ws)

Array([[11., 20., 29.],
       [11., 20., 29.]], dtype=float32)

In [8]:
# vmap 可以指定 in_axes 和 out_axes
auto_batch_convolve_v2 = jax.vmap(convolve, in_axes=1, out_axes=1)

xst = jnp.transpose(xs)
wst = jnp.transpose(ws)

auto_batch_convolve_v2(xst, wst)

Array([[11., 11.],
       [20., 20.],
       [29., 29.]], dtype=float32)

In [11]:
# vmap 可以指定 in_axes 和 out_axes 的不同组合
batch_convolve_v3 = jax.vmap(convolve, in_axes=[0, None])

batch_convolve_v3(xs, w)

Array([[11., 20., 29.],
       [11., 20., 29.]], dtype=float32)

In [12]:
# vmap 可以指定 in_axes 和 out_axes 的再一个例子
# 沿行还是列计算点集？

# 定义一个简单的函数，计算两个向量的点积
def dot_product(x, y):
    return jnp.inner(x, y)

# 输入数据
x = jnp.array([[1, 2, 3], [4, 5, 6]])  # 形状 (2, 3)
y = jnp.array([[7, 8, 9], [10, 11, 12]])  # 形状 (2, 3)

# 使用 vmap，指定 in_axes 和 out_axes
# in_axes=[0, 0] 表示对 x 和 y 的第 0 维进行批处理
# out_axes=0 表示输出的批处理维度保持在第 0 维
batched_dot_v1 = jax.vmap(dot_product, in_axes=[0, 0], out_axes=0)
result = batched_dot_v1(x, y)
print("Result with in_axes=[0, 0], out_axes=0:", result)  # 输出形状 (2,)

# 改变 in_axes 和 out_axes
# in_axes=[1, 1] 表示对 x 和 y 的第 1 维进行批处理
# out_axes=1 表示输出的批处理维度在第 1 维
batched_dot_v2 = jax.vmap(dot_product, in_axes=[1, 1])
result_v2 = batched_dot_v2(x, y)
print("Result with in_axes=[1, 1], out_axes=1:", result_v2)  # 输出形状 (3,)

Result with in_axes=[0, 0], out_axes=0: [ 50 167]
Result with in_axes=[1, 1], out_axes=1: [47 71 99]


In [ ]:
# vmap 可以指定 in_axes 和 out_axes 的再一个例子
# in_axes 可以是 None

# 定义一个简单函数
def add_vectors(x, y):
    return x + y

# 输入数据
x = jnp.array([[1, 2, 3], [4, 5, 6]])  # 形状 (2, 3)
y = jnp.array([10, 20, 30])            # 形状 (3,)

# 使用 vmap，指定 in_axes 和 out_axes
# in_axes=[0, None] 表示对 x 的第 0 维进行批处理，而 y 不进行批处理
# out_axes=1 表示输出的批处理维度在第 1 维
batched_add = jax.vmap(add_vectors, in_axes=[0, None], out_axes=1)

result = batched_add(x, y)
print(result)  # 输出形状 (3, 2)

In [ ]:
# vmap 可以指定 in_axes 和 out_axes 的再一个例子

# 3 个 3x3 矩阵
matrices = jnp.array([
    [[1, 2, 3],
     [4, 5, 6],
     [7, 8, 9]],

    [[10, 11, 12],
     [13, 14, 15],
     [16, 17, 18]],

    [[19, 20, 21],
     [22, 23, 24],
     [25, 26, 27]]
])  # shape (3, 3, 3)

# 3 个向量
vectors = jnp.array([
    [1, 8, 0],
    [1, 2, 0],
    [0, 0, 3]
])  # shape (3, 3)

def matvec_mul(m, v):
    return jnp.matmul(m, v)

batched = jax.vmap(matvec_mul, in_axes=(0, 1), out_axes=0)
result = batched(matrices, vectors)

print("Result shape:", result.shape)  # (3, 3)
print(result)

In [14]:
#上面的例子涉及到了自动广播机制， 实际上 2*3 和 3维向量在 jax 中可以相乘，其实是自动广播了
a = jnp.ones((2, 3))  # shape (2, 3)
b = jnp.ones((3))  # shape (1, 3)
print(jnp.matmul(a, b).shape)  # shape (2,)

# 但是如果我们想要的是 2*3 和 1*3 的矩阵相乘， 却会报错
a = jnp.ones((2, 3))  # shape (2, 3)
b = jnp.ones((1, 3))  # shape (1, 3)
#print(jnp.matmul(a, b))  # 这行代码会报错

(2,)


In [ ]:
# 号外：下面的三个 array 形状当然是不同的
a = jnp.array([1,2,3])
print(a.shape)  # shape (3,)

b = jnp.ones((1, 3))  # shape (1, 3)
print(b.shape)  # shape (1, 3)

c = jnp.zeros((3, 1))  # shape (3, 1)
print(c.shape)  # shape (3, 1)